In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
# import kagglehub
# rounakbanik_the_movies_dataset_path = kagglehub.dataset_download('rounakbanik/the-movies-dataset')

# print('Data source import complete.')


# Data load, clean and format

In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import os
%pip install langid
import langid
import time
import ast
import timeit

%pip install transformers torch
from transformers import pipeline

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 100)

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


/Users/mateo/Library/Python/3.13/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import configparser
config = configparser.ConfigParser()
config_file = "config.ini"
config.read(config_file)

['config.ini']

In [4]:

movies_file = config["FILES"]["MOVIES_FILE"]
ratings_file = config["FILES"]["RATINGS_FILE"]
credits_file = config["FILES"]["CREDITS_FILE"]

movies_df = pd.read_csv(movies_file, low_memory=False)
ratings_df = pd.read_csv(ratings_file)
credits_df = pd.read_csv(credits_file)

OUTPUT_FILE = "output/output.txt"

In [6]:
movies_df.shape

(7990, 24)

In [7]:
ratings_df.shape

(100006, 4)

In [8]:
credits_df.shape

(45476, 3)

In [9]:
movies_df.dtypes

adult                       bool
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
id                         int64
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                    int64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                       bool
vote_average             float64
vote_count                 int64
dtype: object

In [10]:
ratings_df.dtypes

userId         int64
movieId        int64
rating       float64
timestamp      int64
dtype: object

In [11]:
credits_df.dtypes

cast    object
crew    object
id       int64
dtype: object

In [12]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', 'poster_path': '/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg',...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-10-30,373554033,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, 'name': 'Fantasy'}, {'id': 10751, 'name': 'Family'}]",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an enchanted board game that opens the door to a magical w...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'name': 'Teitler Film', 'id': 2550}, {'name': 'Inters...","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-15,262797249,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso_639_1': 'fr', 'name': 'Français'}]",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collection', 'poster_path': '/nLvUdqgPgm3F85NMCii9gVFUcet...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, 'name': 'Comedy'}]",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John...,11.712900,/6ksm1sjKMFLbO7UY2i6G1ju9SML.jpg,"[{'name': 'Warner Bros.', 'id': 6194}, {'name': 'Lancaster Gate', 'id': 19464}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for Love.,Grumpier Old Men,False,6.5,92
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusi...",3.859495,/16XOMpEaLWkrcPqSQqhTmeJuqQl.jpg,"[{'name': 'Twentieth Century Fox Film Corporation', 'id': 306}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-12-22,81452156,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself... and never let you forget it.,Waiting to Exhale,False,6.1,34
4,False,"{'id': 96871, 'name': 'Father of the Bride Collection', 'poster_path': '/nts4iOmNnq7GNicycMJ9pSA...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she'...",8.387519,/e64sOI48hQXyru7naBFyssKFxVd.jpg,"[{'name': 'Sandollar Productions', 'id': 5842}, {'name': 'Touchstone Pictures', 'id': 9195}]","[{'iso_3166_1': 'US', 'name': 'United States of America'}]",1995-02-10,76578911,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's In For The Surprise Of His Life!,Father of the Bride Part II,False,5.7,173


In [13]:
ratings_df.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [14]:
credits_df.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)', 'credit_id': '52fe4284c3a36847f8024f95', 'gender'...","[{'credit_id': '52fe4284c3a36847f8024f49', 'department': 'Directing', 'gender': 2, 'id': 7879, '...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', 'credit_id': '52fe44bfc3a36847f80a7c73', 'gender': ...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'department': 'Production', 'gender': 2, 'id': 511, '...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'credit_id': '52fe466a9251416c75077a8d', 'gender': 2...","[{'credit_id': '52fe466a9251416c75077a89', 'department': 'Directing', 'gender': 2, 'id': 26502, ...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah' Jackson"", 'credit_id': '52fe44779251416c91011aad...","[{'credit_id': '52fe44779251416c91011acb', 'department': 'Directing', 'gender': 2, 'id': 2178, '...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', 'credit_id': '52fe44959251416c75039eb9', 'gender': ...","[{'credit_id': '52fe44959251416c75039ed7', 'department': 'Sound', 'gender': 2, 'id': 37, 'job': ...",11862


In [15]:
movies_df["id"].nunique()

7989

In [16]:
ratings_df["movieId"].nunique()

9068

In [17]:
credits_df["id"].nunique()

45432

## Dataframes Cleaning

In [18]:
movies_df_columns = ["id", "title", "genres", "release_date", "overview", "production_countries", "spoken_languages", "budget", "revenue"]
ratings_df_columns = ["movieId", "rating", "timestamp"]
credits_df_columns = ["id", "cast"]

In [19]:
# Discard unuseful columns
movies_df_cleaned = movies_df.dropna(subset=movies_df_columns)[movies_df_columns].copy()
ratings_df_cleaned = ratings_df.dropna(subset=ratings_df_columns)[ratings_df_columns].copy()
credits_df_cleaned = credits_df.dropna(subset=credits_df_columns)[credits_df_columns].copy()

In [20]:
# Dates Formatting
movies_df_cleaned['release_date'] = pd.to_datetime(movies_df_cleaned['release_date'], format='%Y-%m-%d', errors='coerce')
ratings_df_cleaned['timestamp'] = pd.to_datetime(ratings_df_cleaned['timestamp'], unit='s')

In [21]:
# Numbers Formatting
movies_df_cleaned['budget'] = pd.to_numeric(movies_df_cleaned['budget'], errors='coerce')
movies_df_cleaned['revenue'] = pd.to_numeric(movies_df_cleaned['revenue'], errors='coerce')

In [22]:
# Replace json fields with string arrays
def dictionary_to_list(dictionary_str):
    try:
        dictionary_list = ast.literal_eval(dictionary_str)
        return [data['name'] for data in dictionary_list]
    except (ValueError, SyntaxError):
        return []

movies_df_cleaned['genres'] = movies_df_cleaned['genres'].apply(dictionary_to_list)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].apply(dictionary_to_list)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].apply(dictionary_to_list)
credits_df_cleaned['cast'] = credits_df_cleaned['cast'].apply(dictionary_to_list)

In [23]:
movies_df_cleaned['genres'] = movies_df_cleaned['genres'].astype(str)
movies_df_cleaned['production_countries'] = movies_df_cleaned['production_countries'].astype(str)
movies_df_cleaned['spoken_languages'] = movies_df_cleaned['spoken_languages'].astype(str)

In [24]:
movies_df_cleaned.head()

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue
0,862,Toy Story,"['Animation', 'Comedy', 'Family']",1995-10-30,"Led by Woody, Andy's toys live happily in his room until Andy's birthday brings Buzz Lightyear o...",['United States of America'],['English'],30000000,373554033
1,8844,Jumanji,"['Adventure', 'Fantasy', 'Family']",1995-12-15,When siblings Judy and Peter discover an enchanted board game that opens the door to a magical w...,['United States of America'],"['English', 'Français']",65000000,262797249
2,15602,Grumpier Old Men,"['Romance', 'Comedy']",1995-12-22,A family wedding reignites the ancient feud between next-door neighbors and fishing buddies John...,['United States of America'],['English'],0,0
3,31357,Waiting to Exhale,"['Comedy', 'Drama', 'Romance']",1995-12-22,"Cheated on, mistreated and stepped on, the women are holding their breath, waiting for the elusi...",['United States of America'],['English'],16000000,81452156
4,11862,Father of the Bride Part II,['Comedy'],1995-02-10,"Just when George Banks has recovered from his daughter's wedding, he receives the news that she'...",['United States of America'],['English'],0,76578911


In [25]:
ratings_df_cleaned.head()

,movieId,rating,timestamp
0,31,2.5,2009-12-14 02:52:24
1,1029,3.0,2009-12-14 02:52:59
2,1061,3.0,2009-12-14 02:53:02
3,1129,2.0,2009-12-14 02:53:05
4,1172,4.0,2009-12-14 02:53:25


In [26]:
credits_df_cleaned.head()

,id,cast
0,862,"[Tom Hanks, Tim Allen, Don Rickles, Jim Varney, Wallace Shawn, John Ratzenberger, Annie Potts, J..."
1,8844,"[Robin Williams, Jonathan Hyde, Kirsten Dunst, Bradley Pierce, Bonnie Hunt, Bebe Neuwirth, David..."
2,15602,"[Walter Matthau, Jack Lemmon, Ann-Margret, Sophia Loren, Daryl Hannah, Burgess Meredith, Kevin P..."
3,31357,"[Whitney Houston, Angela Bassett, Loretta Devine, Lela Rochon, Gregory Hines, Dennis Haysbert, M..."
4,11862,"[Steve Martin, Diane Keaton, Martin Short, Kimberly Williams-Paisley, George Newbern, Kieran Cul..."


In [27]:
movies_df_cleaned.dtypes

id                               int64
title                           object
genres                          object
release_date            datetime64[ns]
overview                        object
production_countries            object
spoken_languages                object
budget                           int64
revenue                          int64
dtype: object

In [28]:
ratings_df_cleaned.dtypes

movieId               int64
rating              float64
timestamp    datetime64[ns]
dtype: object

In [29]:
credits_df_cleaned.dtypes

id       int64
cast    object
dtype: object

# Queries a Resolver

- Q1: Peliculas y sus géneros de los años 2000 con producción Argentina y Española.
- Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.
- Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.
- Q4: Top 10 de actores con mayor participación en películas de producción Argentina con
fecha de estreno posterior al 2000
- Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo, para películas de habla inglesa con producción americana, estrenadas a partir del año 2000

### Q1:  Peliculas y sus géneros de los años 00' con producción Argentina y Española Q2:

In [ ]:
movies_argentina_españa_00s_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['production_countries'].str.contains('Spain', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000) &
    (movies_df_cleaned['release_date'].dt.year < 2010)
]

In [ ]:
movies_argentina_españa_00s_df.shape


In [ ]:
movies_argentina_españa_00s_df[["title", "genres"]]

with open(OUTPUT_FILE, "w") as f:
    f.write("query 1\n")
    for index, row in movies_argentina_españa_00s_df.iterrows():
        title = row["title"]
        genres = row["genres"]
        f.write(f"{title},{genres}\n")
    f.write("\n")

In [ ]:
movies_argentina_españa_00s_df[["title", "genres"]]

### Q2: Top 5 de países que más dinero han invertido en producciones sin colaborar con otros países.

In [ ]:
solo_country_df = movies_df_cleaned[movies_df_cleaned['production_countries'].apply(lambda x: len(eval(x)) == 1)]

In [ ]:
solo_country_df.loc[:, 'country'] = solo_country_df['production_countries'].apply(lambda x: eval(x)[0])

In [ ]:
investment_by_country = solo_country_df.groupby('country')['budget'].sum().sort_values(ascending=False)

In [ ]:
top_5_countries = investment_by_country.head(5).reset_index()

print(top_5_countries)

In [ ]:
with open(OUTPUT_FILE, "a") as f:
    f.write("query 2\n")
    for index, row in top_5_countries.iterrows():
        country = row["country"]
        budget = row["budget"]
        f.write(f"{country},{budget}\n")
    f.write("\n")

### Q3: Películas de Producción Argentina estrenadas a partir del 2000, con mayor y con menor promedio de rating.

In [ ]:
movies_argentina_post_2000_df = movies_df_cleaned[
    (movies_df_cleaned['production_countries'].str.contains('Argentina', case=False, na=False)) &
    (movies_df_cleaned['release_date'].dt.year >= 2000)
]

In [ ]:
movies_argentina_post_2000_df = movies_argentina_post_2000_df.astype({'id': int})

In [ ]:
ranking_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(ratings_df_cleaned,
                                                                                left_on="id",
                                                                                right_on="movieId")
mean_ranking_arg_post_2000_df = ranking_arg_post_2000_df.groupby(["id", "title"])['rating'].mean().reset_index()

In [ ]:
title_max = ""
rating_max = ""
title_min = ""
rating_min = ""

if not mean_ranking_arg_post_2000_df['rating'].empty:
    index_max = mean_ranking_arg_post_2000_df['rating'].idxmax()
    index_min = mean_ranking_arg_post_2000_df['rating'].idxmin()
    
    title_max = mean_ranking_arg_post_2000_df.iloc[index_max]["title"]
    rating_max = mean_ranking_arg_post_2000_df.iloc[index_max]["rating"]

    title_min = mean_ranking_arg_post_2000_df.iloc[index_min]["title"]
    rating_min = mean_ranking_arg_post_2000_df.iloc[index_min]["rating"]

with open(OUTPUT_FILE, "a") as f:
    f.write("query 3\n")
    f.write(f"max, {title_max},{rating_max}\n")
    f.write(f"min, {title_min},{rating_min}\n")
    f.write("\n")

### Q4: Top 10 de actores con mayor participación en películas de producción Argentina posterior al 2000

In [ ]:
cast_arg_post_2000_df = movies_argentina_post_2000_df[["id", "title"]].merge(credits_df_cleaned,
                                                                                on="id")
cast_and_movie_arg_post_2000_df = cast_arg_post_2000_df.set_index("id")["cast"].apply(pd.Series).stack().reset_index("id", name="name")
cast_per_movie_quantities = cast_and_movie_arg_post_2000_df.groupby(["name"]).count().reset_index().rename(columns={"id":"count"})
cast_per_movie_quantities.nlargest(10, 'count')

with open(OUTPUT_FILE, "a") as f:
    f.write("query 4\n")
    for index, row in cast_per_movie_quantities.nlargest(10, 'count').iterrows():
        name = row["name"]
        count = row["count"]
        f.write(f"{name},{count}\n")
    f.write("\n")

### Q5: Average de la tasa ingreso/presupuesto de peliculas con overview de sentimiento positivo vs. sentimiento negativo

In [30]:
q5_input_df = movies_df_cleaned.copy()

In [31]:
# Nos quedamos con rows válidas
q5_input_df = q5_input_df.loc[q5_input_df['budget'] != 0]
q5_input_df = q5_input_df.loc[q5_input_df['revenue'] != 0]

In [32]:
q5_input_df.shape

(2368, 9)

In [33]:
# Cargar modelo preentrenado para análisis de sentimiento
sentiment_analyzer = pipeline('sentiment-analysis', model='distilbert-base-uncased-finetuned-sst-2-english', truncation=True)

Device set to use mps:0


In [34]:
start_time = time.time()
q5_input_df['sentiment'] = q5_input_df['overview'].fillna('').apply(lambda x: sentiment_analyzer(x)[0]['label'])
elapsed_time = time.time() - start_time
print(f"Execution time: {elapsed_time:.2f} seconds")

Execution time: 17.74 seconds


In [35]:
q5_input_df["rate_revenue_budget"] = q5_input_df["revenue"] / q5_input_df["budget"]

In [36]:
q5_input_df.sample(10)

,id,title,genres,release_date,overview,production_countries,spoken_languages,budget,revenue,sentiment,rate_revenue_budget
2825,17352,South Pacific,"['Music', 'Romance', 'War']",1958-03-18,Can a girl from Little Rock find happiness with a mature French planter she got to know one ench...,['United States of America'],"['English', 'Français']",6000000,36800000,POSITIVE,6.133333
1831,33667,Gentleman's Agreement,"['Drama', 'Romance']",1947-11-11,A magazine writer poses as a Jew to expose anti-Semitism.,['United States of America'],['English'],2000000,7800000,NEGATIVE,3.900000
5565,47942,When Time Ran Out...,"['Action', 'Adventure', 'Drama', 'Thriller']",1980-03-28,An active volcano threatens a south Pacific island resort and its guests as a power struggle ens...,['United States of America'],['English'],20000000,3763988,NEGATIVE,0.188199
3905,10577,Dracula 2000,"['Thriller', 'Horror']",2000-12-22,"In the millenium version of this classic Gothic horror we find Abraham Van Helsing (Plummer), wh...",['United States of America'],['English'],28000000,33000337,NEGATIVE,1.178583
1546,9437,Kiss the Girls,"['Drama', 'Mystery', 'Thriller', 'Crime']",1997-10-03,Forensic psychologist Alex Cross travels to North Carolina and teams with escaped kidnap victim ...,['United States of America'],"['English', '普通话']",27000000,60527873,POSITIVE,2.241773
6401,9486,Johnny English,"['Adventure', 'Action', 'Comedy']",2003-04-06,Rowan plays the eponymous lead character in a spoof spy thriller. During the course of the story...,"['France', 'United Kingdom']","['日本語', 'English', 'Français']",40000000,160583018,POSITIVE,4.014575
4961,11870,Big Fat Liar,"['Comedy', 'Family']",2002-02-08,"Fourteen-year-old Jason Shepherd (Muniz) has a reputation for stretching the truth. So, when big...",['United States of America'],"['English', 'Français', 'Italiano']",15000000,52970014,NEGATIVE,3.531334
4146,8095,Cleopatra,"['Drama', 'History', 'Romance']",1963-06-12,"Historical epic. The triumphs and tragedy of the Egyptian queen, Cleopatra.The winner of four Os...","['United Kingdom', 'United States of America', 'Switzerland']","['English', 'Português']",31115000,71000000,POSITIVE,2.281858
1551,2649,The Game,"['Drama', 'Thriller', 'Mystery']",1997-09-12,"In honor of his birthday, San Francisco banker Nicholas Van Orton, a financial genius and a cold...",['United States of America'],"['广州话 / 廣州話', 'Deutsch', 'English']",50000000,109423648,NEGATIVE,2.188473
5646,9645,Ghost Ship,"['Horror', 'Mystery', 'Thriller']",2002-10-25,"After discovering a passenger ship missing since 1962 floating adrift on the Bering Sea, salvage...","['Australia', 'United States of America']","['English', 'Italiano', 'Español']",35000000,68349884,NEGATIVE,1.952854


In [38]:
average_rate_by_sentiment = q5_input_df.groupby("sentiment")["rate_revenue_budget"].mean().reset_index()
print(average_rate_by_sentiment)

  sentiment  rate_revenue_budget
0  NEGATIVE         11270.382583
1  POSITIVE          7520.000858


In [ ]:
with open(OUTPUT_FILE, "a") as f:
    f.write("query 5\n")
    for index, row in top_5_countries.iterrows():
        sentiment = row["sentiment"]
        rate_revenue_budget = round(row["rate_revenue_budget"], 4)
        f.write(f"{sentiment}    {rate_revenue_budget}\n")
    f.write("\n")